# Testing basic tools in documentation


[x] Add    
[x] Get


In [1]:
import molsysmt as msm

IndentationError: unexpected indent (to_molsysmt_MolSys.py, line 440)

In [2]:
item = msm.convert('1L2Y', 'mmcif.PDbxContainers.DataContainer')

NameError: name 'msm' is not defined

In [ ]:
molsys = msm.convert(item)

In [ ]:
molsys.topology.chains

In [ ]:
molsys.topology.bonds[0:50]